<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/Model_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam



Using TensorFlow backend.


In [2]:
%%time
# read the data
df = pd.read_excel('data.xlsx')

# creating a subset of the relevant attribute name and then dropping the column
df = df[df['attribute_name'] == 'fit'].drop(columns = ['attribute_name'])

# combining similar category attribute values and removing spaces
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'semifit' if x == 'semi fit' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'fittedtailor' if x == 'fitted tailor' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'semifit' if x == 'semifitte' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'fittedtailor' if x == 'fittedtailore' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'straightregular' if x == 'straight regular' else x)

CPU times: user 8.29 s, sys: 79.1 ms, total: 8.37 s
Wall time: 8.4 s


In [3]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value
4,01E223GDRKR84THXZ54GJEW60Y,vince,cashmere camisole,delicate crochet outlines luxe tank pure cashmere,unknown,semifit
37,01DPGSTG4M1RXB26QMMN0MPPB8,j crew,mockneck sweater supersoft yarn,cozy flattering mockneck sweater feel good loo...,sweater,semifit
48,01DPETFFTSDHQ66H03T0F58NX7,j crew,hanro ultralight spaghetti camisole,supersoft mercerize cotton feel like second sk...,pajama intimate,fittedtailor
70,01DVCRRBK29DWT82DRBVVY544X,mother pearl,NaN,black white lyocell partially conceal snap fas...,clothing top blouse,semifit
71,01DVCRRBK29DWT82DRBVVY544X,mother pearl,net sustain mile faux pearl embellish print ly...,black white lyocell partially conceal snap fas...,clothing top blouse,semifit


In [4]:
# creating a feature combining brand, productname, description and brand category

df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [5]:
# the predictor 'text' is assigned to 'X' and the 'attribute_value' t 'y'
X = df['text'].values
# one-hot-encoding the y variable
y = pd.get_dummies(df['attribute_value']).values

### LSTM

In [6]:
# creating a function to create uni-grams and tokenizing data
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [7]:
# creating a function to create bi-grams and tokenizing data
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [8]:
Tokenizer()

In [9]:
# creating tf-idf vectors from uni-gram encoding
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [10]:
# creating tf-idf vectors from bi-gram encoding
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [11]:
X1.shape

(3464, 155)

In [12]:
# concatenating uni-gram and bi-gram
X = np.concatenate([X1, X2], axis = 1)

In [13]:
# splitting data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [14]:
# balancing the sample sets as the proportion of labels is unbalanced
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [15]:
num_classes = y.shape[1]

In [16]:
# defining LSTM model with sigmoid activation functions for prediction
def define_model_LSTM():
    inputs = Input(shape=(length1+length2,))
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=310, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    out = Activation('softmax')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [17]:
model = define_model_LSTM()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 310)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 310, 100)          482700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 310, 16)           7488      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 310, 5)            85        
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 5)              7755      
_________________________________________________________________
reshape_1 (Reshape)          (None, 5)                 0         
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0   

In [18]:
# fitting data onto model
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/Users/dhwanikapadia/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4172 samples, validate on 1044 samples
Epoch 1/10
4172/4172 [==============================] - 4s 1ms/step - loss: 0.4980 - accuracy: 0.8000 - val_loss: 0.5579 - val_accuracy: 0.8000
Epoch 2/10
4172/4172 [==============================] - 4s 936us/step - loss: 0.4955 - accuracy: 0.8000 - val_loss: 0.5497 - val_accuracy: 0.8000
Epoch 3/10
4172/4172 [==============================] - 4s 934us/step - loss: 0.4948 - accuracy: 0.8000 - val_loss: 0.5562 - val_accuracy: 0.8000
Epoch 4/10
4172/4172 [==============================] - 4s 949us/step - loss: 0.4949 - accuracy: 0.8000 - val_loss: 0.5553 - val_accuracy: 0.8000
Epoch 5/10
4172/4172 [==============================] - 4s 911us/step - loss: 0.4942 - accuracy: 0.8000 - val_loss: 0.5526 - val_accuracy: 0.8000
Epoch 6/10
4172/4172 [==============================] - 4s 933us/step - loss: 0.4944 - accuracy: 0.8000 - val_loss: 0.5512 - val_accuracy: 0.8000
Epoch 7/10
4172/4172 [==============================] - 4s 949us/step - loss: 

In [19]:
# evaluating model
model.evaluate(X_test, y_test)

347/347 [==============================] - 0s 638us/step


[0.48754795355137215, 0.8000001311302185]

In [20]:
#predictions for each class from predicting model
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()

0    2065
2    1399
dtype: int64

In [21]:
# probability of predicting each class for various records
model.predict(X)

array([[0.2500319 , 0.15155238, 0.2760512 , 0.1843861 , 0.13797845],
       [0.2506047 , 0.14626297, 0.27083907, 0.19791245, 0.13438083],
       [0.26671308, 0.1462273 , 0.26894182, 0.18754506, 0.13057283],
       ...,
       [0.26213434, 0.15416183, 0.2610137 , 0.19468541, 0.12800482],
       [0.25776097, 0.14705575, 0.2820309 , 0.19006355, 0.12308883],
       [0.22856206, 0.16427822, 0.28661034, 0.18537323, 0.1351762 ]],
      dtype=float32)

In [22]:
# split of classes
pd.Series(y.argmax(axis = -1)).value_counts()

2    1164
3     910
4     659
0     610
1     121
dtype: int64

### LSTM Glove

In [23]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [24]:
# tokenizing the data and integer encoding it

tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [25]:
# running the tokenizer function on features

X = tokenizer.texts_to_sequences(X)

In [26]:
# getting vocab size and length for input

length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [27]:
# padding the data to make it same size (max size)

X = pad_sequences(X, maxlen=length, padding='post')

In [28]:
# splitting training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [29]:
# balancing the dataset to account for disproportionate of labels

resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [30]:
num_classes = y.shape[1]

In [31]:
# using glove vector to create function that makes word emebeddings

def load_glove_vectors():
    embeddings_index = {}
    with open('glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [32]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [33]:
# creating an LSTM Glove model

def define_model_LSTM_Glove():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(LSTM(64, return_sequences=False, dropout=0.2, recurrent_dropout=0.15, activation = 'relu'))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [34]:
model = define_model_LSTM_Glove()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 155, 100)          482800    
_________________________________________________________________
masking_1 (Masking)          (None, 155, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 155, 64)           42240     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 85        
Total params: 559,189
Trainable params: 76,389
Non-trainable params: 482,800
___________________________________________

In [35]:
# fitting model on training data
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 4073 samples, validate on 1019 samples
Epoch 1/10
4073/4073 [==============================] - 11s 3ms/step - loss: 0.4967 - accuracy: 0.8000 - val_loss: 0.5205 - val_accuracy: 0.8000
Epoch 2/10
4073/4073 [==============================] - 10s 2ms/step - loss: 0.4917 - accuracy: 0.7999 - val_loss: 0.5392 - val_accuracy: 0.8000
Epoch 3/10
4073/4073 [==============================] - 10s 2ms/step - loss: 0.4892 - accuracy: 0.7999 - val_loss: 0.5356 - val_accuracy: 0.8000
Epoch 4/10
4073/4073 [==============================] - 10s 2ms/step - loss: 0.4862 - accuracy: 0.8000 - val_loss: 0.5402 - val_accuracy: 0.8000
Epoch 5/10
4073/4073 [==============================] - 10s 2ms/step - loss: 0.4816 - accuracy: 0.8000 - val_loss: 0.5418 - val_accuracy: 0.8000
Epoch 6/10
4073/4073 [==============================] - 10s 2ms/step - loss: 0.4779 - accuracy: 0.8000 - val_loss: 0.5292 - val_accuracy: 0.8000
Epoch 7/10
4073/4073 [==============================] - 10s 2ms/step - loss: 0.469

In [36]:
# evaluating model
model.evaluate(X_test, y_test)

347/347 [==============================] - 0s 872us/step


[0.46038840938713776, 0.8051872849464417]

In [37]:
# making predictions
results = model.predict(X)

In [38]:
# defining a threshold for prediction
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [39]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [40]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,fittedtailor,oversized,relaxed,semifit,straightregular,attribute_value
0,fittedtailor,,relaxed,semifit,straightregular,"fittedtailor, relaxed, semifit, straightregular"
1,fittedtailor,oversized,relaxed,semifit,straightregular,"fittedtailor, oversized, relaxed, semifit, str..."
2,fittedtailor,oversized,relaxed,semifit,,"fittedtailor, oversized, relaxed, semifit"
3,fittedtailor,,relaxed,semifit,straightregular,"fittedtailor, relaxed, semifit, straightregular"
4,fittedtailor,oversized,relaxed,semifit,straightregular,"fittedtailor, oversized, relaxed, semifit, str..."


In [41]:
results_df['attribute_value'].value_counts()

fittedtailor, relaxed, semifit, straightregular               1441
fittedtailor, oversized, relaxed, semifit, straightregular    1145
relaxed, semifit, straightregular                              352
fittedtailor, oversized, relaxed, semifit                      286
fittedtailor, oversized, semifit                                84
oversized, relaxed, semifit, straightregular                    62
fittedtailor, relaxed, semifit                                  52
fittedtailor, semifit                                           31
relaxed, semifit                                                10
oversized, relaxed, semifit                                      1
Name: attribute_value, dtype: int64

In [42]:
import re 
from gensim.parsing.preprocessing import remove_stopwords
import spacy
import pickle
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('model.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [51]:
def get_fit(brand, product_full_name, description, details, brand_category):
    def clean_text(x):
        try:
            x = re.sub(r'<.*?>', '',x)
            x = re.sub(r"[^A-Za-z0-9(),!?\'\`]", " ", x)
            x = re.sub(r"\'s", " \'s", x)
            x = re.sub(r"\'ve", " \'ve", x)
            x = re.sub(r"n\'t", " n\'t", x)
            x = re.sub(r"\'re", " \'re", x)
            x = re.sub(r"\'d", " \'d", x)
            x = re.sub(r"\'ll", " \'ll", x)
            x = re.sub(r",", " , ", x)
            x = re.sub(r"!", " ! ", x)
            x = re.sub(r"\(", "", x)
            x = re.sub(r"\)", "", x)
            x = re.sub(r"\?", "", x)
            x = re.sub(r"/", "", x)
            x = re.sub(r"\s{2,}", " ", x)
            return x.lower()
        except:
            return ''

    def lemmatizer(x):
        return ' '.join([token.lemma_ for token in nlp(x)])
        
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    with open('model.pickle', 'rb') as handle:
        model = pickle.load(handle)

    data = pd.DataFrame({'brand':brand,'product_full_name':product_full_name,'description':description,'details':details,'brand_category':brand_category},index=[0]) 
    df = data.copy()
    for key in df.columns:
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
        df[key] = df[key].apply(lemmatizer)
        df[key] = df[key].apply(clean_text)
        df[key] = df[key].apply(remove_stopwords)
    df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category'] + ' ' + df['details']).apply(str)
    X = df['text'].values
    X = tokenizer.texts_to_sequences(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    results = model.predict(X)
    results_df = pd.DataFrame()
    results_mask = results > 0.1
    for i in range(len(label_list)):
        results_df[label_list[i]] = results_mask[:,i]
        results_df[label_list[i]] = results_df[label_list[i]].apply(int)
    results_df['sum']  = 0
    for key in label_list:
        results_df['sum'] = results_df['sum'] + results_df[key]
    (results_df['sum'] == 0).sum()
    results_df = results_df.drop(columns = ['sum'])
    for key in label_list:
        results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

    results_df['attribute_value'] = ''
    for key in label_list:
        results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

    results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
    data['attribute_value'] = results_df['attribute_value']
    return data

In [52]:
brand = "frame"
product_full_name = "les second medium noir"
description = "'minimal , modern styling meet refined luxury les second caba tote craft exquisite leather , structured handbag equip adjustable high polish peg buttonhole double drop handle , center welt seam , detachable pouch , frame logo discreetly emboss'"
details = np.nan
brand_category = 'accessory'

In [53]:
get_fit(brand, product_full_name, description, details, brand_category)

,brand,product_full_name,description,details,brand_category,attribute_value
0,frame,les second medium noir,"'minimal , modern styling meet refined luxury ...",NaN,accessory,"fittedtailor, oversized, relaxed, semifit"


In [46]:
results=pd.DataFrame(results)
results.to_excel('results.xlsx')

In [49]:
results_df.to_excel('results_df.xlsx')